### Setup your environnement variables

In [3]:
import os

os.environ["OPENAI_API_KEY"] = "YOUR API KEY"
os.environ["UIFORM_API_KEY"] = "YOUR API KEY" # go to https://uiform.com to create your API Key

# Get started

In [15]:
from dotenv import load_dotenv
assert load_dotenv() # Load environment variables from a .env file

In [16]:
from uiform import AsyncUiForm
uiclient = AsyncUiForm()
await uiclient.models.list()

['claude-3-5-sonnet-latest',
 'claude-3-5-sonnet-20241022',
 'claude-3-opus-20240229',
 'claude-3-sonnet-20240229',
 'claude-3-haiku-20240307',
 'gpt-4o',
 'gpt-4o-mini',
 'chatgpt-4o-latest',
 'gpt-4o-2024-11-20',
 'gpt-4o-2024-08-06',
 'gpt-4o-2024-05-13',
 'gpt-4o-mini-2024-07-18',
 'o1-2024-12-17',
 'o1-mini-2024-09-12',
 'grok-2-vision-1212',
 'grok-2-1212',
 'gemini-2.0-flash-exp',
 'gemini-1.5-flash-8b',
 'gemini-1.5-flash',
 'gemini-1.5-pro']

# Example 1 - Analyze informations from an email

#### Option A - You use our preprocess endpoint and openAI client

In [8]:
import json
from uiform import AsyncUiForm, Schema
from openai import AsyncOpenAI

with open("freight/booking_confirmation_json_schema.json", "r") as f:
    json_schema = json.load(f)


uiclient = AsyncUiForm()
doc_msg = await uiclient.documents.create_messages(
    document = "freight/booking_confirmation.jpg",
)

schema_obj = Schema(
    json_schema = json_schema
)


# Now you can use your favorite model to analyze your document
client = AsyncOpenAI()
completion = await client.chat.completions.create(
    model="gpt-4o-mini", 
    messages=schema_obj.openai_messages + doc_msg.openai_messages,
    response_format=schema_obj.response_format_json
)

# Validate the response against the original schema if you want to remove the reasoning fields
assert completion.choices[0].message.content is not None
extraction = schema_obj.pydantic_model.model_validate_json(
    completion.choices[0].message.content
)

print(extraction)

thinking='Analysis shows:\n- Confirmed booking document from ACME Corporation\n- Contains detailed shipment and transport terms\n- Client identified as ACME Corporation with VAT number GB123456789\n- Total price of 1500 EUR excluding taxes clearly stated\n- Shipment contains hazardous materials, specifically UN1202.\n- Pickup and delivery dates and times outlined clearly.\n- Important transport requirements and conditions laid out clearly in the document. Verifies that all required documentation and inspections are needed during transport.' booking_id='SHIP-001' payment={'total_price': 1500, 'currency': 'EUR'} client={'company_name': 'ACME Corporation', 'SIREN': None, 'VAT_number_thinking': 'VAT number is identified as GB123456789 which is a valid format for VAT registration in the UK.\nFound VAT number in the header of the document next to ACME Corporation.', 'VAT_number': 'GB123456789', 'city': 'Manchester', 'postal_code': 'M1 4WP', 'country': 'GB', 'code': None, 'email': 'client@acm

#### Option B - Using UiForm `extract` endpoint

In [10]:
import json
from uiform.client import AsyncUiForm

uiclient = AsyncUiForm()
completion = await uiclient.documents.extractions.parse(
    json_schema = "freight/booking_confirmation_json_schema.json",
    document = "freight/booking_confirmation.jpg",
    model="gpt-4o-mini",
    temperature=0
)

print(completion.choices[0].message.parsed)

{'thinking': 'Analysis shows:\n- Clear booking confirmation document from ACME Corporation\n- Contains 1 distinct shipment under single booking reference\n- Client identified as ACME Corporation from letterhead\n- Total value and payment terms clearly stated\n- All required signatures present', 'booking_id': 'SHIP-001', 'payment': {'total_price': 1500, 'currency': 'EUR'}, 'client': {'company_name': 'ACME Corporation', 'SIREN': 'GB123456789', 'VAT_number': 'GB123456789', 'city': 'London', 'postal_code': 'WC2N 5DU', 'country': 'GB', 'email': 'client@acme.com'}, 'number_of_shipments': 1, 'shipments': [{'shipment_id': 'SHIP-001', 'sender': {'company_name': 'ACME Corporation', 'address': {'line1': '123 Elm Street', 'line2': 'Suite 500', 'city': 'London', 'postal_code': 'WC2N 5DU', 'country': 'GB'}, 'phone_number': '+44 20 7946 0958', 'email_address': 'client@acme.com', 'pickup_datetime': {'date': '2023-02-05', 'start_time': '08:00', 'end_time': '12:00'}, 'observations': 'Pickup at Acme Corp

## Appendix A - Use text as modality

In [11]:
from uiform.client import AsyncUiForm

uiclient = AsyncUiForm()

completion = await uiclient.documents.extractions.parse(
    json_schema = "freight/booking_confirmation_json_schema.json",
    document="freight/booking_confirmation.jpg",
    model="gpt-4o-mini-2024-07-18",
    temperature=0,
    modality='text'
)

print(completion.choices[0].message.parsed)

{'thinking': 'Analysis shows:\n- Clear booking confirmation document from ACME Corporation\n- Client identified as ACME Corporation with VAT number GB123456789\n- Contains 1 distinct shipment under booking reference SHIP-001\n- Total price of the booking is 1500 EUR, excluding taxes\n- All required signatures and contact details present', 'booking_id': 'SHIP-001', 'payment': {'total_price': 1500, 'currency': 'EUR'}, 'client': {'company_name': 'ACME Corporation', 'SIREN': None, 'VAT_number_thinking': 'Found VAT number GB123456789 in the header of the document.', 'VAT_number': 'GB123456789', 'city': 'Manchester', 'postal_code': 'M1 4WP', 'country': 'GB', 'code': None, 'email': 'client@acme.com'}, 'number_of_shipments': 1, 'shipments_thinking': 'Shipment Analysis:\n- Shipment identified by reference number SHIP-001\n- Pickup from ACME Corporation, Manchester to Beta Industries, Munich\n- Delivery scheduled between 5/2/2023 and 5/3/2023\n- Weight: 500 kg, Volume: 1.44 m3, 5 pallets\n- Temp

## Appendix B - Add additional context

In [12]:
from uiform.client import AsyncUiForm

uiclient = AsyncUiForm()

text_operations = {'regex_instructions' : [
        {
            "name": "vat_number",
            "pattern": r"[Ff][Rr]\s*(\d\s*){11}",
            "description": "VAT number in the format XX999999999"
        }
    ]
}

completion = await uiclient.documents.extractions.parse(
    json_schema = "freight/booking_confirmation_json_schema.json",
    document = "freight/booking_confirmation.jpg",
    model="gpt-4o-mini-2024-07-18",
    temperature=0,
    text_operations=text_operations
)

print(completion.choices[0].message.parsed)

{'thinking': 'Analysis shows:\n- Clear booking confirmation document from ACME Corporation\n- Contains 1 distinct shipment under single booking reference\n- Client identified as ACME Corporation from letterhead\n- Total value and payment terms clearly stated\n- All required signatures present', 'booking_id': 'SHIP-001', 'payment': {'total_price': 1500, 'currency': 'EUR'}, 'client': {'company_name': 'ACME Corporation', 'SIREN': 'FR12 345 678 901', 'VAT_number': 'GB123456789', 'city': 'London', 'postal_code': 'WC2N 5DU', 'country': 'GB', 'email': 'client@acme.com'}, 'number_of_shipments': 1, 'shipments': [{'shipment_id': 'SHIP-001', 'sender': {'company_name': 'ACME Corporation', 'address': {'line1': '123 Elm Street', 'line2': 'Suite 500', 'city': 'London', 'postal_code': 'WC2N 5DU', 'country': 'GB'}, 'phone_number': '+44 20 7946 0958', 'email_address': 'client@acme.com', 'pickup_datetime': {'date': '2023-05-02', 'start_time': '08:00', 'end_time': '12:00'}, 'observations': 'Pickup at Acme